In [ ]:
import os
import re
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from google.colab import drive, userdata

# file management
drive.mount('/content/drive')
WORK_DIR = '/content/drive/MyDrive/Projects/skillextraction'

# work dir shortcut function
def work_dir(*args):
    return os.path.join(WORK_DIR, *args)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load english sentence dataset from article
article_en = pd.read_csv(work_dir('Data', 'dataset.csv'))
article_en = article_en.groupby('skill').head(10)

# check
print(article_en.shape)
print(article_en['skill'].value_counts().value_counts())
article_en.head(3)

(138260, 2)
count
10    13826
Name: count, dtype: int64


,sentence,skill
0,the ideal candidate for this position should b...,advise customers on sewing patterns
1,we need an employee who is able to assist our ...,advise customers on sewing patterns
2,if you possess good communication skills and h...,advise customers on sewing patterns


In [ ]:
# stitch up danish translations of dataset from article
article_da = pd.concat([pd.read_csv(work_dir('Translated_data', s)) for s in os.listdir(work_dir('Translated_data')) if re.match(r'^translated_sentences\_[0-9]+\.csv$', s)])
article_da = article_da.groupby('skill').head(10)

# check
print(article_da.shape)
print(article_da['skill'].value_counts().value_counts())
article_da.head(3)

(138170, 2)
count
10    13798
9        18
1         5
7         1
6         1
5         1
3         1
2         1
Name: count, dtype: int64


,skill,sentence
0,procurement legislation,en omfattende forståelse af indkøbslovgivning ...
1,procurement legislation,kendskab og erfaring med indkøbslovgivning er ...
2,procurement legislation,ansøgere med erfaring inden for områder relate...


In [ ]:
# get replacements for leftovers
leftovers = pd.read_csv(work_dir('Translated_data', 'leftover_translated_sentences_0.csv'))

# check
print(leftovers.shape)
print(leftovers['skill'].value_counts().value_counts())
leftovers.head(3)

(280, 2)
count
10    28
Name: count, dtype: int64


,skill,sentence
0,Wireshark,ønske om at ansætte en erfaren person med erfa...
1,Wireshark,kun kandidater med en grundig forståelse af Wi...
2,Wireshark,Som netværkssikkerhedsanalytiker vil du anvend...


In [ ]:
# combine main translations with leftover replacement (and ignore previous)
article_da = pd.concat([leftovers, article_da], ignore_index=True).groupby('skill').head(10)

# check
print(article_da.shape)
print(article_da['skill'].value_counts().value_counts())
article_da.head(3)

(138260, 2)
count
10    13826
Name: count, dtype: int64


,skill,sentence
0,Wireshark,ønske om at ansætte en erfaren person med erfa...
1,Wireshark,kun kandidater med en grundig forståelse af Wi...
2,Wireshark,Som netværkssikkerhedsanalytiker vil du anvend...


In [ ]:
# stitch up extra danish dataset
extra_da = pd.concat([pd.read_csv(work_dir('Data', s)) for s in os.listdir(work_dir('Data')) if re.match(r'^sentences\_[0-9]+\.csv$', s)])
extra_da = extra_da.groupby('conceptUri').head(15)

# check
print(extra_da.shape)
print(extra_da['conceptUri'].value_counts().value_counts())
extra_da.head(3)

(208320, 2)
count
15    13885
14        2
9         1
8         1
Name: count, dtype: int64


,conceptUri,completion
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Vi søger en erfaren musikalsk leder til at til...
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,Som leder af musikalsk personale vil du være a...
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,Den ideelle kandidat vil have ekspertise inden...


In [ ]:
# get esco english/danish
esco_en = pd.read_csv(work_dir('ESCO', 'ESCO dataset - v1.1.2 - classification - en - csv', 'skills_en.csv'))
esco_da = pd.read_csv(work_dir('ESCO', 'ESCO dataset - v1.1.2 - classification - da - csv', 'skills_da.csv'))

# limit to necessary and merge for easier mapping
esco = pd.merge(
    esco_en[['conceptUri', 'preferredLabel', 'description', 'altLabels']],
    esco_da[['conceptUri', 'preferredLabel', 'description', 'altLabels']],
    on='conceptUri',
    suffixes=('_en', '_da')
)

# check
print(esco.shape)
print(esco['preferredLabel_en'].value_counts().value_counts())
print(esco['preferredLabel_da'].value_counts().value_counts())
print(esco['description_en'].value_counts().value_counts())
print(esco['description_da'].value_counts().value_counts())
esco.head(3)

(13896, 7)
count
1    13896
Name: count, dtype: int64
count
1    13879
2        8
Name: count, dtype: int64
count
1    13890
2        3
Name: count, dtype: int64
count
1    13894
2        1
Name: count, dtype: int64


,conceptUri,preferredLabel_en,description_en,altLabels_en,preferredLabel_da,description_da,altLabels_da
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,Assign and manage staff tasks in areas such as...,manage staff of music\ncoordinate duties of mu...,lede musikalsk personale,Tildele og forvalte personaleopgaver på område...,NaN
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,Supervise the operations of a correctional fac...,oversee prison procedures\nmanage correctional...,føre tilsyn med fængselsprocedurer,Føre tilsyn med driften af et fængsel eller an...,NaN
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,"Identify oppression in societies, economies, c...",apply non-oppressive practices\napply an anti-...,anvende antioppressiv praksis,"Identificere undertrykkelse i samfund, økonomi...",NaN


In [ ]:
# get rid of nans and duplicates (potential duplicate proxies!!!)
esco = esco.dropna(subset=['preferredLabel_da']) \
           .dropna(subset=['description_en']) \
           .dropna(subset=['description_da']) \
           .drop_duplicates(subset=['preferredLabel_da']) \
           .drop_duplicates(subset=['description_en']) \
           .drop_duplicates(subset=['description_da'])

# check
print(esco.shape)
print(esco['preferredLabel_en'].value_counts().value_counts())
print(esco['preferredLabel_da'].value_counts().value_counts())
print(esco['description_en'].value_counts().value_counts())
print(esco['description_da'].value_counts().value_counts())

(13884, 7)
count
1    13884
Name: count, dtype: int64
count
1    13884
Name: count, dtype: int64
count
1    13884
Name: count, dtype: int64
count
1    13884
Name: count, dtype: int64


In [ ]:
# filter esco for what we have synthetic sentences (just ignore rest for now)
esco = esco[esco['preferredLabel_en'].isin(article_en['skill'])]

# check
print(esco.shape)
esco.head(3)

(13813, 7)


,conceptUri,preferredLabel_en,description_en,altLabels_en,preferredLabel_da,description_da,altLabels_da
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,Assign and manage staff tasks in areas such as...,manage staff of music\ncoordinate duties of mu...,lede musikalsk personale,Tildele og forvalte personaleopgaver på område...,NaN
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,Supervise the operations of a correctional fac...,oversee prison procedures\nmanage correctional...,føre tilsyn med fængselsprocedurer,Føre tilsyn med driften af et fængsel eller an...,NaN
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,"Identify oppression in societies, economies, c...",apply non-oppressive practices\napply an anti-...,anvende antioppressiv praksis,"Identificere undertrykkelse i samfund, økonomi...",NaN


In [ ]:
# filter synthetic sentences for what we have esco! (less in danish?)
article_en = article_en[article_en['skill'].isin(esco['preferredLabel_en'])]
article_da = article_da[article_da['skill'].isin(esco['preferredLabel_en'])]
extra_da = extra_da[extra_da['conceptUri'].isin(esco['conceptUri'])]

# check
print(article_en.shape)
print(article_da.shape)
print(extra_da.shape)
print(article_en['skill'].value_counts().value_counts())
print(article_da['skill'].value_counts().value_counts())
print(extra_da['conceptUri'].value_counts().value_counts())
article_en.head(3)
article_da.head(3)
extra_da.head(3)

(138130, 2)
(138130, 2)
(207165, 2)
count
10    13813
Name: count, dtype: int64
count
10    13813
Name: count, dtype: int64
count
15    13808
14        2
9         1
8         1
Name: count, dtype: int64


,conceptUri,completion
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,Vi søger en erfaren musikalsk leder til at til...
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,Som leder af musikalsk personale vil du være a...
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,Den ideelle kandidat vil have ekspertise inden...


In [ ]:
# unfold the esco alt labels
alts_en = esco[['conceptUri', 'altLabels_en']].rename(columns={'altLabels_en': 'sentence'}) \
                                              .set_index(['conceptUri'])['sentence'] \
                                              .str.split('\n') \
                                              .explode() \
                                              .reset_index() \
                                              .dropna()

alts_da = esco[['conceptUri', 'altLabels_da']].rename(columns={'altLabels_da': 'sentence'}) \
                                              .set_index(['conceptUri'])['sentence'] \
                                              .str.split('\n') \
                                              .explode() \
                                              .reset_index() \
                                              .dropna()

# check
print(alts_en['conceptUri'].value_counts().value_counts())
print(alts_da['conceptUri'].value_counts().value_counts())
alts_en

count
5     2049
6     2045
8     1993
7     1937
9     1267
4      787
10     705
11     613
12     581
1      536
3      368
2      243
13     237
14      97
15      61
16      29
17      21
18      16
20      11
19       7
22       3
32       3
24       2
46       1
21       1
25       1
26       1
30       1
35       1
38       1
40       1
45       1
56       1
Name: count, dtype: int64
count
1     314
5     187
4     123
3      87
6      83
2      55
7      28
8       6
9       2
17      1
13      1
10      1
Name: count, dtype: int64


,conceptUri,sentence
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage staff of music
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,coordinate duties of musical staff
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage music staff
3,http://data.europa.eu/esco/skill/0005c151-5b5a...,direct musical staff
4,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff
...,...,...
97035,http://data.europa.eu/esco/skill/fff74a70-4f82...,position guardrails and toeboards
97036,http://data.europa.eu/esco/skill/fff74a70-4f82...,fit guardrails and toeboards
97037,http://data.europa.eu/esco/skill/fff74a70-4f82...,fitting of guardrails and toeboards
97038,http://data.europa.eu/esco/skill/fff74a70-4f82...,guardrail and toeboard attachment


In [ ]:
# stitch up danish translations of english alt labels
alts_en_trans = pd.concat([pd.read_csv(work_dir('Translated_data', s)) for s in os.listdir(work_dir('Translated_data')) if re.match(r'^translated\_alts\_[0-9]+\.csv$', s)])
alts_en_trans = alts_en_trans[alts_en_trans['conceptUri'].isin(esco['conceptUri'])].dropna()

# check
print(alts_en_trans.shape)
print(alts_en_trans['conceptUri'].value_counts().value_counts())
alts_en_trans

(92722, 2)
count
10    2389
5     2050
6     2043
8     1993
7     1935
9     1267
4      788
1      535
3      368
2      242
16       3
11       3
12       2
20       1
18       1
14       1
Name: count, dtype: int64


,conceptUri,sentence
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,lede musikpersonale
1,http://data.europa.eu/esco/skill/0005c151-5b5a...,koordinere opgaver for musikpersonale
2,http://data.europa.eu/esco/skill/0005c151-5b5a...,lede musikstaben
3,http://data.europa.eu/esco/skill/0005c151-5b5a...,instrukt musikpersonalet
4,http://data.europa.eu/esco/skill/0005c151-5b5a...,lede det musikalske personale
...,...,...
24,http://data.europa.eu/esco/skill/fff74a70-4f82...,fiksering af rækværk og fodlister
25,http://data.europa.eu/esco/skill/fff74a70-4f82...,placere rækværk og fodlister
26,http://data.europa.eu/esco/skill/fff74a70-4f82...,montere rækværk og fodlister
27,http://data.europa.eu/esco/skill/fff74a70-4f82...,montering af rækværk og fodlister


In [ ]:
# map label to conceptUri
map = dict(zip(esco['preferredLabel_en'], esco['conceptUri']))

# combine it all
skills = pd.concat([
    esco[['conceptUri', 'preferredLabel_en']].rename(columns={'preferredLabel_en': 'sentence'}).assign(group=1),
    esco[['conceptUri', 'preferredLabel_da']].rename(columns={'preferredLabel_da': 'sentence'}).assign(group=2),
    esco[['conceptUri', 'description_en']].rename(columns={'description_en': 'sentence'}).assign(group=3),
    esco[['conceptUri', 'description_da']].rename(columns={'description_da': 'sentence'}).assign(group=4),
    article_en[['sentence']].assign(conceptUri=article_en['skill'].map(map)).assign(group=5),
    article_da[['sentence']].assign(conceptUri=article_da['skill'].map(map)).assign(group=6),
    extra_da[['conceptUri', 'completion']].rename(columns={'completion': 'sentence'}).assign(group=7), # very slightly unbalanced
    alts_en[['conceptUri', 'sentence']].assign(group=8), # unbalanced
    alts_en_trans[['conceptUri', 'sentence']].assign(group=9), # unbalanced
    alts_da[['conceptUri', 'sentence']].assign(group=10) # unbalanced
])

# check
print(skills.shape)
print(skills['conceptUri'].value_counts().value_counts())
skills.sample(10)

(731159, 3)
count
51     2017
55     1979
53     1966
49     1786
57     1223
47      695
59      684
60      616
61      594
41      530
45      297
62      247
39      192
43      146
52      134
54      118
63      106
48       84
50       83
64       66
56       50
58       49
65       30
46       25
66       24
67       17
69       10
44        7
68        7
42        4
70        4
73        3
81        3
72        2
71        2
105       1
100       1
74        1
75        1
77        1
79        1
84        1
86        1
87        1
93        1
94        1
95        1
26        1
Name: count, dtype: int64


,conceptUri,sentence,group
31461,http://data.europa.eu/esco/skill/8e1dd4a6-0d0e...,hvis du er dygtig til at skabe skæreplaner og ...,6
804,http://data.europa.eu/esco/skill/dcace5e5-9b8d...,Vi søger en kvalificeret kandidat til at tage ...,7
25223,http://data.europa.eu/esco/skill/428b92d4-d13a...,animal by-products processing,8
131,http://data.europa.eu/esco/skill/8794f8b6-8217...,produktfjernelse fra forme,9
12426,http://data.europa.eu/esco/skill/e5d4da88-aac9...,udarbejde godsforsendelsesrapporter,2
113,http://data.europa.eu/esco/skill/8ce4a4af-969b...,"plant træer, buske og hække",9
23258,http://data.europa.eu/esco/skill/3cc36ca3-61fd...,chiropody principles,8
129875,http://data.europa.eu/esco/skill/5f1e9642-588c...,et kompetent niveau af forståelse for talt ita...,6
7966,http://data.europa.eu/esco/skill/2be699a7-416a...,we need someone who can navigate the complexit...,5
4075,http://data.europa.eu/esco/skill/4b205f0c-0278...,Instruct students in the theory and practice o...,3


In [ ]:
# save!
skills.to_json(work_dir('Data', 'skills.json'), orient='records', lines=True, index=False)

In [ ]:
# sanity check
df = pd.read_json(work_dir('Data', 'skills.json'), orient='records', lines=True)
print(df.shape)
df.head(3)

(731159, 3)


,conceptUri,sentence,group
0,http://data.europa.eu/esco/skill/0005c151-5b5a...,manage musical staff,1
1,http://data.europa.eu/esco/skill/00064735-8fad...,supervise correctional procedures,1
2,http://data.europa.eu/esco/skill/000709ed-2be5...,apply anti-oppressive practices,1
